In [ ]:
!pip install statsmodels


In [4]:
import pandas as pd
import numpy as np
from scipy import stats 
import statsmodels.api as sm

## 3.1 Data Simulation

In [ ]:
#seteo básico
np.random.seed(42)
n = 1000

X1 = np.random.normal(loc=15, scale=5, size=n)  # Continua
X2 = np.random.normal(loc=10, scale=2, size=n)  # Continua
X3 = np.random.binomial(n=1, p=0.6, size=n)     # Binaria
X4 = np.random.binomial(n=1, p=0.25, size=n)    # Binaria

# Generamos la asignación al tratamiento D ~ Bern(0.5)
D = np.random.binomial(n=1, p=0.5, size=n)
#print(D)

epsilon = np.random.normal(loc=0, scale=1, size=n) 
Y = 2*D + 0.5*X1 - 0.3*X2 + 0.2*X3 + epsilon
#print(epsilon)

In [ ]:
#creamos dataframe
data = {
    'Y': Y,
    'D': D,
    'X1': X1,
    'X2': X2,
    'X3': X3,
    'X4': X4
}
df = pd.DataFrame(data)

#df.head()

In [ ]:
# Separamos los datos en grupo de tratamiento y control para la comparación
treatment_group = df[df['D'] == 1]
control_group = df[df['D'] == 0]

#print(treatment_group)
#print(control_group)


In [ ]:
covariates = ['X1', 'X2', 'X3', 'X4']

# Iteramos sobre cada covariable para realizar el t-test
for cov in covariates:
    # Comparamos la media de la covariable 'cov' en el grupo de tratamiento vs. control
    t_stat, p_value = stats.ttest_ind(treatment_group[cov], control_group[cov])
    
    print(f"Covariable: {cov}")
    print(f"  Media Grupo Tratamiento: {treatment_group[cov].mean():.4f}")
    print(f"  Media Grupo Control:     {control_group[cov].mean():.4f}")
    print(f"  P-value:                 {p_value:.4f}")
    
    if p_value > 0.05:
        print("  No hay diferencia estadística. grupos  balanceados")
    else:
        print("   Hay una diferencia estadística. grupos NO  balanceados")
    print("-" * 50)

## 3.2 Estimating the Average Treatment Effect

In [ ]:
#Estimate the treatment effect (ATE) using a simple reg:
y = df['Y']
X_simple = df[['D']]
#con constante:
X_simple = sm.add_constant(X_simple) 
model_simple = sm.OLS(y, X_simple).fit()
print(model_simple.summary())



In [ ]:
# estimo ATE controlando por todas las covariables
X_multiple = df[['D', 'X1', 'X2', 'X3', 'X4']]
#AGREGO LA CONSTANTE:
X_multiple = sm.add_constant(X_multiple) 
model_multiple = sm.OLS(y, X_multiple).fit()
print(model_multiple.summary())

                                 OLS Regression Results                                
Dep. Variable:                      Y   R-squared (uncentered):                   0.976
Model:                            OLS   Adj. R-squared (uncentered):              0.976
Method:                 Least Squares   F-statistic:                              8017.
Date:                Mon, 22 Sep 2025   Prob (F-statistic):                        0.00
Time:                        22:38:27   Log-Likelihood:                         -1401.2
No. Observations:                1000   AIC:                                      2812.
Df Residuals:                     995   BIC:                                      2837.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

Los valores del ATE no varian mucho, los valores que tomó fue de 2.2014 y  2.0606 para el modelo simple y para el modelo múltiple respectivamente. Mientras que el valor verdadero es 2. Esto se debe a que la asignación del D fue aletoria y no esta correlacionada con las X.

Acerca de lo que pasa con los errores estandares es que bajaron de 0.173 para el modelo simple y 0.062 el modelo multiple. Lo que pasa es que las covariables son buenas predictoras del Y (asi es nuestro PGD), por lo que al incluirlos reduicmos la magnitud del error residual.

## 3.3 LASSO and Variable Selection (3 points)

### 3.3.1 Use LASSO to select covariates that predict Y (1 point)

In [26]:
from sklearn.linear_model import LassoCV

In [ ]:
#definimos las covariables
X_lasso = df.drop(columns=['Y', 'D'])
y_lasso = df['Y']
lasso_cv = LassoCV(cv=10, random_state=42).fit(X_lasso, y_lasso)

coefficients = lasso_cv.coef_
selected_mask = coefficients != 0  #variables eleggidas osn las que son dif de cero
selected_covariates = X_lasso.columns[selected_mask].tolist()
#print(selected_covariates)

#variables selecionadas
print(f"El alpha (lambda_min) óptimo encontrado por CV es: {lasso_cv.alpha_:.4f}")
print("\nCoeficientes de LASSO para cada covariable:")
for name, coef in zip(X_lasso.columns, coefficients):
    print(f"  {name}: {coef:.4f}")

print(f"\n✅ Covariables seleccionadas por LASSO: {selected_covariates}")


['X1', 'X2', 'X3']


In [36]:
#reestimando ATE con las varibles seleccionadas por LASSO
X_post_lasso = df[['D'] + selected_covariates]
y = df['Y'] 

# Agregamos la constante y ajustamos el modelo
X_post_lasso = sm.add_constant(X_post_lasso)
model_post_lasso = sm.OLS(y, X_post_lasso).fit()
print(model_post_lasso.summary())



                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.889
Model:                            OLS   Adj. R-squared:                  0.888
Method:                 Least Squares   F-statistic:                     1990.
Date:                Mon, 22 Sep 2025   Prob (F-statistic):               0.00
Time:                        22:49:40   Log-Likelihood:                -1399.4
No. Observations:                1000   AIC:                             2809.
Df Residuals:                     995   BIC:                             2833.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4175      0.199     -2.094      0.0

La precicisón de Lasso es casi identica a usar el modelo completo, el coeficiente de LASSO del D es 2.0565 y el de completo es 2.0432, mientras que el error estandar es el mismo para ambos 0.062. Esto se debe a que Lasso hizo un buen trabajo eliminando X4 que era variable irrelevante del modelo (recordemos que el PGD no depende de X4), es decir logra la casi la misma precisión que el modelo completo sin necesidad de agregar variables innecesarias